In [1]:
from typing import Dict, Any
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.porter import PorterStemmer
import re

# Carga los datos una sola vez al iniciar la aplicación
dfETL = pd.read_csv('movies_ETL.csv', parse_dates=['release_date'])


def cantidad_filmaciones_mes(mes: str):
    data = dfETL

    meses = {
        'enero': 1,
        'febrero': 2,
        'marzo': 3,
        'abril': 4,
        'mayo': 5,
        'junio': 6,
        'julio': 7,
        'agosto': 8,
        'septiembre': 9,
        'octubre': 10,
        'noviembre': 11,
        'diciembre': 12
    }

    num_mes = meses.get(mes.lower())

    if num_mes is None:
        raise ValueError('Nombre de mes inválido')

    filmaciones_mes = data[data['release_date'].dt.month == num_mes]
    cantidad_filmaciones = len(filmaciones_mes)

    return f" mes: {mes.capitalize()}, cantidad: {cantidad_filmaciones}"

def cantidad_filmaciones_dia(dia: str):
    data = dfETL

    data['release_date'] = pd.to_datetime(data['release_date'])

    dias = {
        'lunes': 0,
        'martes': 1,
        'miercoles': 2,
        'jueves': 3,
        'viernes': 4,
        'sabado': 5,
        'domingo': 6
    }

    num_dia = dias.get(dia.lower())

    if num_dia is None:
        raise ValueError('Nombre de día inválido')

    filmaciones_dia = data[data['release_date'].dt.weekday == num_dia]
    cantidad_filmaciones = len(filmaciones_dia)

    return {f" dia:{dia.capitalize()}, cantidad: {cantidad_filmaciones} "}
    
def score_titulo(titulo: str):
    data = dfETL

    titulo = titulo.title()

    match = data[data['title'] == titulo]

    if not match.empty:
        titulo1 = match['title'].values[0]
        anio = match['release_year'].values[0]
        score_popularity = match['popularity'].values[0]

        score_popularity = round(score_popularity, 2)

        return {f"titulo: {titulo1}, año: {anio}, popularidad: {score_popularity}"}
        
    return {"mensaje": "No se encontró la película especificada."}
    

def votos_titulo(titulo: str):
    data = dfETL
    data['title'] = data['title'].str.title()
    titulo = titulo.title()

    if data['title'].str.contains(titulo).any():
        if data[data['title'] == titulo]['vote_count'].item() < 2000:
            return {"mensaje": "La película no cuenta con al menos 2000 valoraciones."}
        else:
            titulo1 = titulo
            anio = data[data['title'] == titulo]['release_year'].item()
            c_votos = data[data['title'] == titulo]['vote_count'].item()
            score = data[data['title'] == titulo]['vote_average'].item()

            c_votos = int(c_votos)


            return { f"titulo: {titulo1}, año: {anio}, voto_total: {c_votos} , voto_promedio de {score}"}
            
    else:
        return {"mensaje": "La película no se encontró."}


def get_actor(nombre_actor: str):
    data = dfETL
    nombre_a = str(nombre_actor).title()

    if data['name'].apply(lambda x: isinstance(x, list) and nombre_a in x).any():
        c_filmaciones = data[data['name'].apply(lambda x: isinstance(x, list) and nombre_a in x)]['title'].count()
        retorno = data[data['name'].apply(lambda x: isinstance(x, list) and nombre_a in x)]['retorno_inversion'].sum()
        promedio = data[data['name'].apply(lambda x: isinstance(x, list) and nombre_a in x)]['retorno_inversion'].mean()

        retorno = round(retorno, 2)
        promedio = round(promedio, 2)

        return {f" actor: {nombre_a}, cantidad_filmaciones: {c_filmaciones}, retorno_total: {retorno}, voto_promedio: {promedio}"}
        

    elif data['name'].apply(lambda x: isinstance(x, str) and nombre_a in x).any():
        c_filmaciones = data[data['name'].apply(lambda x: isinstance(x, str) and nombre_a in x)]['title'].count()
        retorno = data[data['name'].apply(lambda x: isinstance(x, str) and nombre_a in x)]['retorno_inversion'].sum()
        promedio = data[data['name'].apply(lambda x: isinstance(x, str) and nombre_a in x)]['retorno_inversion'].mean()

        retorno = round(retorno, 2)
        promedio = round(promedio, 2)

        return {f" actor: {nombre_a}, cantidad_filmaciones: {c_filmaciones}, retorno_total: {retorno}, voto_promedio: {promedio}"}
        #return {"mensaje": f"El actor {nombre_a} ha participado en {c_filmaciones} filmaciones, ha conseguido un retorno total de {retorno} con un promedio de {promedio} por filmación."}
    else:
        return {"mensaje": f"El actor {nombre_a} no se encuentra en la base de datos."}


from typing import Dict, Any

def get_director(nombre_director: str) -> Dict[str, Any]:
    director = dfETL[dfETL['Director'] == nombre_director]

    if director.empty:
        return {"mensaje": "El director no se encuentra en el dataset."}

    exito_director = director.shape[0]

    # Ajustar la columna "release_date" para eliminar la hora
    director['release_date'] = director['release_date'].dt.date

    peliculas_director = director[['title', 'release_date', 'budget', 'revenue']]
    peliculas_director['release_date'] = peliculas_director['release_date'].astype(str)

    return {
        "director": nombre_director,
        "retorno_director": exito_director,
        "peliculas_director": peliculas_director.to_dict(orient='records')
        }
 

In [2]:
get_director('Steven Spielberg')

C:\Users\USER\AppData\Local\Temp\ipykernel_10644\2559447937.py:148: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  director['release_date'] = director['release_date'].dt.date
C:\Users\USER\AppData\Local\Temp\ipykernel_10644\2559447937.py:151: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  peliculas_director['release_date'] = peliculas_director['release_date'].astype(str)


{'director': 'Steven Spielberg',
 'retorno_director': 31,
 'peliculas_director': [{'title': 'Jurassic Park',
   'release_date': '1993-06-11',
   'budget': 63000000,
   'revenue': 920100000},
  {'title': "Schindler's List",
   'release_date': '1993-11-29',
   'budget': 22000000,
   'revenue': 321365567},
  {'title': 'E.T. the Extra-Terrestrial',
   'release_date': '1982-04-03',
   'budget': 10500000,
   'revenue': 792965326},
  {'title': 'Raiders of the Lost Ark',
   'release_date': '1981-06-12',
   'budget': 18000000,
   'revenue': 389925971},
  {'title': 'Indiana Jones and the Last Crusade',
   'release_date': '1989-05-24',
   'budget': 48000000,
   'revenue': 474171806},
  {'title': 'Jaws',
   'release_date': '1975-06-18',
   'budget': 7000000,
   'revenue': 470654000},
  {'title': 'The Lost World: Jurassic Park',
   'release_date': '1997-05-23',
   'budget': 73000000,
   'revenue': 229074524},
  {'title': 'Amistad',
   'release_date': '1997-12-03',
   'budget': 36000000,
   'revenue

In [3]:

from typing import Dict, Any

def get_director(nombre_director: str) -> Dict[str, Any]:
    director = dfETL[dfETL['Director'] == nombre_director]

    if director.empty:
        return {"mensaje": "El director no se encuentra en el dataset."}

    exito_director = director.shape[0]

    # Ajustar la columna "release_date" para eliminar la hora
    director['release_date'] = director['release_date'].dt.date

    peliculas_director = director[['title', 'release_date', 'budget', 'revenue']]
    peliculas_director['release_date'] = peliculas_director['release_date'].astype(str)

    return {
        "director": nombre_director,
        "retorno_director": exito_director,
        "peliculas_director": peliculas_director.to_dict(orient='records')
    }
   


In [4]:
get_director('Steven Spielberg')

C:\Users\USER\AppData\Local\Temp\ipykernel_10644\2532092046.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  director['release_date'] = director['release_date'].dt.date
C:\Users\USER\AppData\Local\Temp\ipykernel_10644\2532092046.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  peliculas_director['release_date'] = peliculas_director['release_date'].astype(str)


{'director': 'Steven Spielberg',
 'retorno_director': 31,
 'peliculas_director': [{'title': 'Jurassic Park',
   'release_date': '1993-06-11',
   'budget': 63000000,
   'revenue': 920100000},
  {'title': "Schindler's List",
   'release_date': '1993-11-29',
   'budget': 22000000,
   'revenue': 321365567},
  {'title': 'E.T. the Extra-Terrestrial',
   'release_date': '1982-04-03',
   'budget': 10500000,
   'revenue': 792965326},
  {'title': 'Raiders of the Lost Ark',
   'release_date': '1981-06-12',
   'budget': 18000000,
   'revenue': 389925971},
  {'title': 'Indiana Jones and the Last Crusade',
   'release_date': '1989-05-24',
   'budget': 48000000,
   'revenue': 474171806},
  {'title': 'Jaws',
   'release_date': '1975-06-18',
   'budget': 7000000,
   'revenue': 470654000},
  {'title': 'The Lost World: Jurassic Park',
   'release_date': '1997-05-23',
   'budget': 73000000,
   'revenue': 229074524},
  {'title': 'Amistad',
   'release_date': '1997-12-03',
   'budget': 36000000,
   'revenue

In [5]:
get_actor('Tom Cruise')

{' actor: Tom Cruise, cantidad_filmaciones: 47, retorno_total: 183.05, voto_promedio: 3.89'}

In [6]:
votos_titulo('Toy Story')

{'titulo: Toy Story, año: 1995, voto_total: 5415 , voto_promedio de 7.7'}

In [7]:
score_titulo('Toy Story')

{'titulo: Toy Story, año: 1995, popularidad: 21.95'}

In [8]:
cantidad_filmaciones_dia('Lunes')

{' dia:Lunes, cantidad: 3509 '}

In [9]:
cantidad_filmaciones_mes('Enero')

' mes: Enero, cantidad: 5920'

******

In [10]:
import pandas as pd
import numpy as np
import seaborn as snb
import ast # Ayuda a descubrir la gramatica
import datetime
import sklearn
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import difflib

In [11]:
movie_name='Iron Man'

In [12]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import difflib

dfr = pd.read_csv('movies_ETL.csv', parse_dates=['release_date'])

def recommend(movie_name: str):
    id_unique = range(1, len(dfr) + 1)
    dfr['ID'] = list(id_unique)
    dfr1 = dfr[['genres', 'tagline', 'name', 'Director']]
    dfr1 = dfr1.fillna('')
    combined_features = dfr1['genres'] + ' ' + dfr1['tagline'] + ' ' + dfr1['name'] + ' ' + dfr1['Director']

    vectorizer = TfidfVectorizer()
    feature_vectors = vectorizer.fit_transform(combined_features)
    similarity = cosine_similarity(feature_vectors[:25000, :])

    lista_de_todas_las_peliculas = dfr['title'].tolist()
    longitud = len(lista_de_todas_las_peliculas)
    find_close_match = difflib.get_close_matches(movie_name, lista_de_todas_las_peliculas)
    close_match = find_close_match[0]
    index_of_the_movie = dfr[dfr['title'] == close_match]['ID'].values[0]
    similarity_score = list(enumerate(similarity[index_of_the_movie]))
    sorted_similar_movies = sorted(similarity_score, key=lambda x: x[1], reverse=True)
    i = 1

    recommended_movies = []
    for movie_index, similarity_score in sorted_similar_movies:
        if i >= 10:
            break
        if movie_index != index_of_the_movie:
            matching_rows = dfr[dfr['ID'] == movie_index]
            if len(matching_rows) > 0:
                title_from_index = matching_rows['title'].values[0]
                recommended_movies.append(title_from_index)
                i += 1

    return {"mensaje": f"Las películas recomendadas para {movie_name} son:", "recomendaciones": recommended_movies}


In [13]:
recommend('Iron Man')

{'mensaje': 'Las películas recomendadas para Iron Man son:',
 'recomendaciones': ['Winged Migration',
  'G-Men',
  'The Key',
  'Breath',
  'Bukowski - Born into This',
  'The Dangerous Lives of Altar Boys',
  'Deep Water',
  'The Three Faces of Eve',
  'The Killing of John Lennon']}

*************

In [14]:
# Ejemplo de uso de las funciones en Jupyter Notebook
print(cantidad_filmaciones_mes('enero'))
print(cantidad_filmaciones_dia('lunes'))
print(score_titulo('The Godfather'))
print(votos_titulo('The Godfather'))
print(get_actor('Tom Cruise'))
print(get_director('Steven Spielberg'))
print(recommend('The Godfather'))

 mes: Enero, cantidad: 5920
{' dia:Lunes, cantidad: 3509 '}
{'titulo: The Godfather, año: 1972, popularidad: 41.11'}
{'titulo: The Godfather, año: 1972, voto_total: 6024 , voto_promedio de 8.5'}
{' actor: Tom Cruise, cantidad_filmaciones: 47, retorno_total: 183.05, voto_promedio: 3.89'}
{'director': 'Steven Spielberg', 'retorno_director': 31, 'peliculas_director': [{'title': 'Jurassic Park', 'release_date': '1993-06-11', 'budget': 63000000, 'revenue': 920100000}, {'title': "Schindler'S List", 'release_date': '1993-11-29', 'budget': 22000000, 'revenue': 321365567}, {'title': 'E.T. The Extra-Terrestrial', 'release_date': '1982-04-03', 'budget': 10500000, 'revenue': 792965326}, {'title': 'Raiders Of The Lost Ark', 'release_date': '1981-06-12', 'budget': 18000000, 'revenue': 389925971}, {'title': 'Indiana Jones And The Last Crusade', 'release_date': '1989-05-24', 'budget': 48000000, 'revenue': 474171806}, {'title': 'Jaws', 'release_date': '1975-06-18', 'budget': 7000000, 'revenue': 4706540

C:\Users\USER\AppData\Local\Temp\ipykernel_10644\2532092046.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  director['release_date'] = director['release_date'].dt.date
C:\Users\USER\AppData\Local\Temp\ipykernel_10644\2532092046.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  peliculas_director['release_date'] = peliculas_director['release_date'].astype(str)


{'mensaje': 'Las películas recomendadas para The Godfather son:', 'recomendaciones': ['Stars', '20 Dates', 'Moll Flanders', 'Pom Poko', 'Larks on a String', 'Sommer der Gaukler', 'Who Killed the Electric Car?', 'A Murder of Crows', 'Demonic Toys']}
